In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_nl3])),
                     'biases':tf.Variable(tf.random_normal(n_nodes_hl3))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'biases':tf.Variable(tf.random_normal(n_classes))}
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output


In [3]:
from tensorflow.python.ops import rnn, rnn_cell

hm_epochs = 3
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float')

In [4]:
def recurrent_neural_network(x):
    layer = {'weights': tf.Variable(tf.random_normal([rnn_size, n_classes])),
            'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    print("*", x.shape)
    x = tf.transpose(x, [1, 0, 2])
    print("*", x.shape)
    x = tf.reshape(x, [-1, chunk_size])
    print("*", x.shape)
    x = tf.split(x, n_chunks, 0)
    print("*", x)
    
    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=True)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output= tf.matmul(outputs[-1], layer['weights']) + layer['biases']
    
    return output

In [5]:

def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                # print(epoch_x.shape) #(128, 784)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                # print(epoch_x.shape)#(128, 28, 28)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                
            print('Epoch', epoch, 'completed out of ', hm_epochs, 'loss:', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))
        
train_neural_network(x)

* (?, 28, 28)
* (28, ?, 28)
* (?, 28)
* [<tf.Tensor 'split:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:2' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:3' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:4' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:5' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:6' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:7' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:8' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:9' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:10' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:11' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:12' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:13' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:14' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:15' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:16' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:17' shape=(?, 28) dtype=float32>, <tf.Tensor 'split:18' shape=(?, 2